In [3]:
!pip install findspark
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=cbdc5e067332e4a5daba6c964a3786274708643771bef24e16998a3606e9fded
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=ef5d8e4c1afc86fb765e6553c0f7d4d9ce2f40d2a00541791e0c96f5c6f18b80
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [5]:
import pyspark
import findspark
import spark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [27]:
jugadores = spark.read.option('inferSchema', 'true').option('header', 'true').csv('players.csv')
apariciones = spark.read.option('inferSchema', 'true').option('header', 'true').csv('appearances.csv')
competiciones = spark.read.option('inferSchema', 'true').option('header', 'true').csv('competitions.csv')
juegos = spark.read.option('inferSchema', 'true').option('header', 'true').csv('games.csv')

In [15]:
jugadores.groupBy('country_of_birth').agg(countDistinct('player_id').alias('conteo'))\
.orderBy(desc('conteo'))\
.filter(col('country_of_birth').isNotNull())\
.limit(3).show()

+----------------+------+
|country_of_birth|conteo|
+----------------+------+
|          France|  1694|
|           Spain|  1388|
|           Italy|  1312|
+----------------+------+



In [24]:
jugadores.join(apariciones, apariciones.player_id == jugadores.player_id)\
.filter(col('game_id').isNotNull())\
.filter(col('red_cards') > 0)\
.groupBy('pretty_name')\
.agg(count('red_cards').alias('tarjetas_rojas'))\
.orderBy(desc('tarjetas_rojas'))\
.show(truncate=False)

+----------------------+--------------+
|pretty_name           |tarjetas_rojas|
+----------------------+--------------+
|Marcelo               |6             |
|Domenico Berardi      |5             |
|Rodrigo               |5             |
|Granit Xhaka          |5             |
|Thomas Lam            |5             |
|Mario Balotelli       |5             |
|Hilton                |5             |
|Joao Pedro            |4             |
|David Luiz            |4             |
|Rafik Halliche        |4             |
|Kalidou Koulibaly     |4             |
|Ante Rebic            |4             |
|Yannick Cahuzac       |4             |
|Damien Da Silva       |4             |
|Ramon Leeuwin         |4             |
|Timothee Kolodziejczak|4             |
|Roberto Soriano       |4             |
|Rodrigo De Paul       |4             |
|Ivan Ordets           |4             |
|Stefan Mitrovic       |4             |
+----------------------+--------------+
only showing top 20 rows



In [31]:
juegos.join(competiciones, competiciones.competition_id == juegos.competition_code)\
.filter(col('name') == 'premier-liga')\
.groupBy('name').agg(count('game_id').alias('juegos'))\
.show()

+------------+------+
|        name|juegos|
+------------+------+
|premier-liga|  3084|
+------------+------+



In [37]:
juegos.join(competiciones, competiciones.competition_id == juegos.competition_code)\
.groupBy('name').agg(sum('attendance').alias('asistencia'))\
.filter(col('name').isNotNull())\
.orderBy(desc('asistencia'))\
.limit(3)\
.show()

+--------------+----------+
|          name|asistencia|
+--------------+----------+
|premier-league|  86964852|
|    bundesliga|  78102473|
|        laliga|  62943533|
+--------------+----------+



In [39]:
pelis = spark.read.option('inferSchema','true').option('delimiter','|').option('header','true').csv('movies.csv')

+-----------------+--------------------+----+
|            actor|            pelicula| año|
+-----------------+--------------------+----+
|McClure, Marc (I)|       Freaky Friday|2003|
|McClure, Marc (I)|        Coach Carter|2005|
|McClure, Marc (I)|         Superman II|1980|
|McClure, Marc (I)|           Apollo 13|1995|
|McClure, Marc (I)|            Superman|1978|
|McClure, Marc (I)|  Back to the Future|1985|
|McClure, Marc (I)|Back to the Futur...|1990|
|Cooper, Chris (I)|  Me, Myself & Irene|2000|
|Cooper, Chris (I)|         October Sky|1999|
|Cooper, Chris (I)|              Capote|2005|
|Cooper, Chris (I)|The Bourne Supremacy|2004|
|Cooper, Chris (I)|         The Patriot|2000|
|Cooper, Chris (I)|            The Town|2010|
|Cooper, Chris (I)|          Seabiscuit|2003|
|Cooper, Chris (I)|      A Time to Kill|1996|
|Cooper, Chris (I)|Where the Wild Th...|2009|
|Cooper, Chris (I)|         The Muppets|2011|
|Cooper, Chris (I)|     American Beauty|1999|
|Cooper, Chris (I)|             Sy

In [40]:
pelis.groupBy('actor').agg(count('pelicula').alias('conteo')).orderBy(desc('conteo')).show()

+-------------------+------+
|              actor|conteo|
+-------------------+------+
|   Tatasciore, Fred|    38|
|      Welker, Frank|    38|
| Jackson, Samuel L.|    32|
|      Harnell, Jess|    31|
|        Damon, Matt|    27|
|      Willis, Bruce|    27|
|  Cummings, Jim (I)|    26|
|         Hanks, Tom|    25|
|   Lynn, Sherry (I)|    25|
|    McGowan, Mickie|    25|
|    Bergen, Bob (I)|    25|
|      Proctor, Phil|    24|
|   Wilson, Owen (I)|    23|
|        Cruise, Tom|    23|
|         Pitt, Brad|    23|
|Freeman, Morgan (I)|    22|
|Williams, Robin (I)|    22|
|       Depp, Johnny|    22|
|     Morrison, Rana|    22|
|      Diaz, Cameron|    21|
+-------------------+------+
only showing top 20 rows



In [61]:
pelis.select(
    'año',
    when((col('año') > 1900) & (col('año') <= 2000), 'Siglo XX').otherwise('Siglo XXI').alias('siglo')
)\
.groupBy('año', 'siglo').agg(count_distinct('año').alias('conteo'))\
.orderBy(desc('conteo'))\
.show()

+----+---------+------+
| año|    siglo|conteo|
+----+---------+------+
|2012|Siglo XXI|     1|
|2000| Siglo XX|     1|
|1996| Siglo XX|     1|
|2010|Siglo XXI|     1|
|1982| Siglo XX|     1|
|1992| Siglo XX|     1|
|1994| Siglo XX|     1|
|1975| Siglo XX|     1|
|1997| Siglo XX|     1|
|1961| Siglo XX|     1|
|1986| Siglo XX|     1|
|1983| Siglo XX|     1|
|1980| Siglo XX|     1|
|2005|Siglo XXI|     1|
|2003|Siglo XXI|     1|
|1977| Siglo XX|     1|
|2006|Siglo XXI|     1|
|1999| Siglo XX|     1|
|2002|Siglo XXI|     1|
|2007|Siglo XXI|     1|
+----+---------+------+
only showing top 20 rows



In [51]:
ratings = spark.read.option('inferSchema','true').option('header','true').option('delimiter','|').csv('movie_ratings.csv')
ratings.show()

+----------+--------------------+----+
|valoracion|            pelicula| año|
+----------+--------------------+----+
|    1.6339|'Crocodile' Dunde...|1988|
|    7.6177|                  10|1979|
|    1.2864|10 Things I Hate ...|1999|
|    0.3243|           10,000 BC|2008|
|    0.3376|      101 Dalmatians|1996|
|    0.5218|      102 Dalmatians|2000|
|   12.8205|                1066|2012|
|    0.6829|                  12|2007|
|    7.4061|           12 Rounds|2009|
|    2.3677|           127 Hours|2010|
|    1.3585|      13 Going on 30|2004|
|    8.4034|     13 game sayawng|2006|
|      0.59|                1408|2007|
|    4.4292|          15 Minutes|2001|
|    2.2118|           16 Blocks|2006|
|    1.0491|            17 Again|2009|
|    3.9265|                1941|1979|
|   10.4757|2 Days in the Valley|1996|
|       0.4|    2 Fast 2 Furious|2003|
|   11.1111|              2 Guns|2013|
+----------+--------------------+----+
only showing top 20 rows



In [63]:
from pyspark.sql.window import Window
window_spec = Window.partitionBy('año').orderBy(desc('valoracion'))
ratings.withColumn('rn', row_number().over(window_spec))\
.filter(col('rn') == 1)\
.select('año', 'pelicula', 'valoracion')\
.show(truncate=False)

+----+-------------------------------+----------+
|año |pelicula                       |valoracion|
+----+-------------------------------+----------+
|1937|Snow White and the Seven Dwarfs|2.2207    |
|1939|The Wizard of Oz               |7.9215    |
|1940|Pinocchio                      |7.8557    |
|1942|Bambi                          |1.5053    |
|1946|Song of the South              |7.602     |
|1950|Cinderella                     |9.4226    |
|1953|Peter Pan                      |5.4756    |
|1954|Rear Window                    |10.7625   |
|1955|Lady and the Tramp             |5.1258    |
|1956|Around the World in Eighty Days|14.0607   |
|1959|Sleeping Beauty                |6.3919    |
|1960|Psycho                         |10.6375   |
|1961|One Hundred and One Dalmatians |0.6726    |
|1962|The Longest Day                |12.8866   |
|1963|It's a Mad Mad Mad Mad World   |6.626     |
|1964|My Fair Lady                   |7.587     |
|1965|Doctor Zhivago                 |4.9304    |
